In [16]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import sys
import os
os.chdir('..')
sys.path.insert(0, os.getcwd())
import numpy as np

from AE_pipeline_pytorch import (
    LSTMAutoencoder,
    BiLSTMAutoencoder,
    
    GaitBatchIterable,

    train_autoencoder,
    evaluate_and_detect,
    extract_and_save_latents,
    reconstruct_and_evaluate 
)

import math
import torch
from time import time
from torch.utils.data import Dataset, DataLoader
import zarr


2025-06-27 12:13:55.551948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751037235.569827   87609 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751037235.575001   87609 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751037235.587917   87609 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751037235.587935   87609 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751037235.587937   87609 computation_placer.cc:177] computation placer alr

Using device: cuda


In [2]:
print(os.getcwd())

/home/dmartinez/Gait-Stability


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [4]:
#Data loader for Zarr datasets
batch_size = 400
train_loader = DataLoader(GaitBatchIterable("train_cycles.zarr",batch_size=batch_size, return_meta=False),
    batch_size=None,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=2,
)


val_loader = DataLoader(GaitBatchIterable("val_cycles.zarr",batch_size=batch_size, return_meta=False),
    batch_size=None,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=2,
)

test_loader = DataLoader(
    GaitBatchIterable("test_cycles.zarr", batch_size=batch_size, return_meta=True), 
    batch_size=None, 
    num_workers=8,   
    shuffle=False,   
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=2,
)


In [5]:
#To check the data loader
batch = next(iter(train_loader))
for i, item in enumerate(batch):
    print(f"Item {i}: type={type(item)}, shape={getattr(item, 'shape', None)}")

Item 0: type=<class 'torch.Tensor'>, shape=torch.Size([1000, 100, 321])
Item 1: type=<class 'torch.Tensor'>, shape=torch.Size([1000, 100, 321])


In [ ]:
#Train 

model = LSTMAutoencoder(n_timesteps=100, n_vars=321, latent_dim=128).to(device)


# Compila el modelo (PyTorch 2.0 JIT), **antes** de entrenar
if hasattr(torch, "compile"):
    model = torch.compile(model)


# Tu función de train_autoencoder acepta train_loader y val_loader
train_autoencoder(
    model,
    train_loader,
    val_loader,
    run_id="Torch1",
    epochs=20,
    debug=False,
    debug_batches=60
)

Epoch 1/20: train_loss=0.9471, val_loss=0.9669


/home/dmartinez/miniconda3/envs/inv_Di/lib/python3.10/site-packages/torch/utils/data/dataloader.py:750: UserWarning: Length of IterableDataset <AE_pipeline_pytorch.GaitBatchIterable object at 0x746afc2bec50> was reported to be 486(when accessing len(dataloader)), but 487 samples have been fetched. For multiprocessing data-loading, this could be caused by not properly configuring the IterableDataset replica at each worker. Please see https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset for examples.
  warnings.warn(warn_msg)
/home/dmartinez/miniconda3/envs/inv_Di/lib/python3.10/site-packages/torch/utils/data/dataloader.py:750: UserWarning: Length of IterableDataset <AE_pipeline_pytorch.GaitBatchIterable object at 0x746afc2be3e0> was reported to be 105(when accessing len(dataloader)), but 106 samples have been fetched. For multiprocessing data-loading, this could be caused by not properly configuring the IterableDataset replica at each worker. Please see https://pyt

Epoch 2/20: train_loss=0.7191, val_loss=0.7061
Epoch 3/20: train_loss=0.5566, val_loss=0.6503
Epoch 4/20: train_loss=0.5148, val_loss=0.6196
Epoch 5/20: train_loss=0.4853, val_loss=0.5975
Epoch 6/20: train_loss=0.4637, val_loss=0.5813
Epoch 7/20: train_loss=0.4457, val_loss=0.5682
Epoch 8/20: train_loss=0.4314, val_loss=0.5599
Epoch 9/20: train_loss=0.4192, val_loss=0.5478
Epoch 10/20: train_loss=0.4105, val_loss=0.5436
Epoch 11/20: train_loss=0.4003, val_loss=0.5327
Epoch 12/20: train_loss=0.3919, val_loss=0.5270
Epoch 13/20: train_loss=0.3846, val_loss=0.5202
Epoch 14/20: train_loss=0.3775, val_loss=0.5142
Epoch 15/20: train_loss=0.3714, val_loss=0.5089
Epoch 16/20: train_loss=0.3670, val_loss=0.5051
Epoch 17/20: train_loss=0.3613, val_loss=0.5003
Epoch 18/20: train_loss=0.3568, val_loss=0.4976
Epoch 19/20: train_loss=0.3519, val_loss=0.4931


In [ ]:
model.eval()
mse = torch.nn.MSELoss(reduction='none')

all_scores = []
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        recon = model(x)                       # (B, T, 321)
        # Si quieres comparar recon vs y, quizá tu función `evaluate_and_detect`
        # espera (x, y) y devuelve errores. Sino:
        # ejemplo de score: media de MSE por variable
        loss = mse(recon, x).mean(dim=(1,2))   # reconstrucción de x vs x
        # o si tu supervisión es recon vs y:
        # loss = mse(recon, y).mean(dim=(1,2))
        all_scores.append(loss.cpu())

all_scores = torch.cat(all_scores)
print("Test MSE (batch-wise):", all_scores.mean().item())
